In [1]:
import pandas as pd

In [5]:
# trans = pd.read_csv('2018-09-04-trans-eQTLsFDR-CohortInfoRemoved-BonferroniAdded.txt', sep='\t')

In [101]:
sig_trans = pd.read_csv('2018-09-04-trans-eQTLsFDR0.05-CohortInfoRemoved-BonferroniAdded.txt', sep='\t')

In [123]:
def rename_columns(df):
    df= df.rename(columns= {"#CHROM":'chr_name',
                            "POS":"chr_position",
                            "ALT":"effect_allele", 
                            "REF":"other_allele",
                            "BETA":"effect_weight"})
    return(df)

In [66]:
def drop_duplicated_variants(df):
    
    ''' drops variants that comes from different studies, variants that have different alternatives but same rsids,
              and variants that have same postion reference and alternative, but different rsid. 
    returns df with dropped duplicated variants, and the number of SNPs that have switched REF and ALT and their Pos '''
    
    # drop variants that comes from different studies 
    df=df.drop_duplicates(subset=['ID','POS','REF','ALT','#CHROM']).reset_index(drop =True)  
    
    # drop variants that have different alternatives
    df=df.drop_duplicates(subset=['ID','POS','REF','#CHROM']).reset_index(drop =True) 
    
    # drop variants that have same postion and alternative, but different ID
    df=df.drop_duplicates(subset=['POS','REF','ALT','#CHROM']).reset_index(drop =True)
    
    #check everithing is unique 
    q = df.groupby(['#CHROM','POS']).size().reset_index(name='Freq').sort_values(by='Freq',ascending=False)
    print ("we have " + str(len(q.loc[q.Freq == 2].POS)) + " variants that have switched REF and ALT")
    
    return(df, q.loc[q.Freq > 1].POS)

In [102]:
sig_trans = sig_trans.sort_values(by="Pvalue").reset_index(drop=True)

In [103]:
sig_trans = sig_trans.rename(columns= { "SNPChr":"#CHROM",
                                        "SNPPos":"POS",
                                        "SNP":"ID",
                                        "AssessedAllele":"ALT", 
                                        "OtherAllele":"REF",
                                        "Pvalue":"BETA"
                                      })

In [105]:
sig_trans = sig_trans[['#CHROM'  ,'POS' ,'ID'  , 'REF' ,    'ALT',  'BETA']]

In [106]:
sig_trans.loc[:, ('BETA')] = -np.log10(sig_trans['BETA'])

/tmp/ipykernel_628459/1600328138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sig_trans.loc[:, ('BETA')] = -np.log10(sig_trans['BETA'])


In [109]:
sig_trans

,#CHROM,POS,ID,REF,ALT,BETA
0,1,248039451,rs3811444,C,T,307.947346
1,3,56849749,rs1354034,C,T,307.947346
2,12,69732105,rs4761234,T,C,307.947346
3,3,56849749,rs1354034,C,T,307.947346
4,3,56865776,rs12485738,G,A,307.947346
...,...,...,...,...,...,...
59781,6,31240721,rs114415823,C,T,5.080321
59782,9,136942136,rs10821556,C,A,5.080321
59783,12,112486818,rs17696736,A,G,5.080321
59784,6,31379115,rs116204553,G,A,5.080321


In [112]:
sig_trans_qtl,pos = drop_duplicated_variants(sig_trans)

we have 0 variants that have switched REF and ALT


In [122]:
sig_trans_qtl['#CHROM'].unique()

array([ 1,  3, 12,  7,  2,  6, 17, 10,  4, 11,  5, 19, 15,  9, 20, 14,  8,
       21, 16, 22, 13, 18])

In [115]:
!mkdir dataset_trans_qtls

In [116]:
path ='/dh-projects/uk_bb_intergenics/analysis/development/aboughas/results/dataset_trans_qtls/subsets_pgs'

In [117]:
sig_trans_qtl.to_csv(path+'/variant_subset.100k.tsv',sep ='\t', index = False )

In [124]:
sig_trans_qtl = rename_columns( sig_trans_qtl)

In [125]:
sig_trans_qtl.to_csv('dataset_trans_qtls/dataset_trans_qtls.txt',sep ='\t', index = False )